In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show, show_hist
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import glob
import scipy
from osgeo import gdal
import os
from math import pi
from scipy import ndimage

DEM = rasterio.open(r'C:\\Users\\Aubre_M\\Documents\\Fall 2020\\GIS Programming_ Automation\\geog_4092\\Week 8\\data\\data\\bigElk_dem.tif')
Arry_DEM = DEM.read(1)
Fire_per = rasterio.open(r'C:\\Users\\Aubre_M\\Documents\\Fall 2020\\GIS Programming_ Automation\\geog_4092\\Week 8\\data\\data\\fire_perimeter.tif')
Arry_Fire = Fire_per.read(1) 
f = glob.glob(r'C:\\Users\\Aubre_M\\Documents\\Fall 2020\\GIS Programming_ Automation\\geog_4092\\Week 8\\data\\data\\L5_big_elk\\*.tif')

In [6]:
"""
Author: Originally created by Galen Maclaurin, updated by Ricardo Oliveira
Created: Created on 3.15.16, updated on 10.17.19
Purpose: Helper functions to get started with Lab 5
"""
def slopeAspect(dem, cs):
    """Calculates slope and aspect using the 3rd-order finite difference method
    Parameters
    ----------
    dem : numpy array
        A numpy array of a DEM
    cs : float
        The cell size of the original DEM
    Returns
    -------
    numpy arrays
        Slope and Aspect arrays
    """
    kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    dzdx = ndimage.convolve(dem, kernel, mode='mirror') / (8 * cs)
    dzdy = ndimage.convolve(dem, kernel.T, mode='mirror') / (8 * cs)
    slp = np.arctan((dzdx ** 2 + dzdy ** 2) ** 0.5) * 180 / pi
    ang = np.arctan2(-dzdy, dzdx) * 180 / pi
    aspect = np.where(ang > 90, 450 - ang, 90 - ang)
    return slp, aspect

def reclassAspect(npArray):
    """Reclassify aspect array to 8 cardinal directions (N,NE,E,SE,S,SW,W,NW),
    encoded 1 to 8, respectively (same as ArcGIS aspect classes).
    Parameters
    ----------
    npArray : numpy array
        numpy array with aspect values 0 to 360
    Returns
    -------
    numpy array
        numpy array with cardinal directions
    """
    return np.where((npArray > 22.5) & (npArray <= 67.5), 2,
    np.where((npArray > 67.5) & (npArray <= 112.5), 3,
    np.where((npArray > 112.5) & (npArray <= 157.5), 4,
    np.where((npArray > 157.5) & (npArray <= 202.5), 5,
    np.where((npArray > 202.5) & (npArray <= 247.5), 6,
    np.where((npArray > 247.5) & (npArray <= 292.5), 7,
    np.where((npArray > 292.5) & (npArray <= 337.5), 8, 1)))))))

def reclassByHisto(npArray, bins):
    """Reclassify np array based on a histogram approach using a specified
    number of bins. Returns the reclassified numpy array and the classes from
    the histogram.
    Parameters
    ----------
    npArray : numpy array
        Array to be reclassified
    bins : int
        Number of bins
    Returns
    -------
    numpy array
        umpy array with reclassified values
    """
    histo = np.histogram(npArray, bins)[1]
    rClss = np.zeros_like(npArray)
    for i in range(bins):
        rClss = np.where((npArray >= histo[i]) & (npArray <= histo[i + 1]),
                         i + 1, rClss)
    return rClss

def zonal_stats(zones, value_raster, file_name):
    s = {'Zone' :[], 'Min': [], 'Max':[], 'Mean':[], 'Stdv':[], 'Count': []}
    for u in np.unique(zones):
        ras = np.where(zones==u, u, np.nan)
        s['Min'].append(np.nanmin(ras * value_raster))
        s['Max'].append(np.nanmax(ras * value_raster))
        s['Mean'].append(np.nanmean(ras * value_raster))
        s['Stdv'].append(np.nanstd(ras * value_raster))
        s['Count'].append(np.where(zones == u, 1, 0).sum())
        s['Zone'].append(int(u))
    df = pd.DataFrame(s)
    df.to_csv(file_name)
    return df



B3 = glob.glob(r'./data/data/L5_big_elk/*B3.tif')
B4 = glob.glob(r'./data/data/L5_big_elk/*B4.tif')

healthy_area = np.where(Arry_Fire == 2)
burned_area = np.where(Arry_Fire  ==1)
# Lists
M_CO_R = []
Recov_R = []
ys = []
for  r in f:
    if 'B3' in r:
        with rasterio.open(r) as data:
            ys.append(r[-11:-7])
    

#Part I
for x,y in zip(B3, B4):
    RED = rasterio.open(x,'r').read(1)
    NearIR = rasterio.open(y,'r').read(1)
    NDVI = ((NearIR-RED)/(NearIR+RED))
    NDVI_m = NDVI[healthy_area].mean()
    recovery_R = NDVI/NDVI_m
    Burn_m = recovery_R[burned_area].mean()
    M_CO_R.append(Burn_m)
    flat = recovery_R.ravel()
    Recov_R.append(flat)
stacked_Rr = np.vstack(Recov_R)
Fit_line = np.polyfit(range(10), stacked_Rr, 1)[0]
line_Rshape = Fit_line.reshape(280, 459)
mean_CO = np.where(Arry_Fire ==1, line_Rshape, np.nan)
print('The coeficient of recovery across all years was', np.nanmean(mean_CO))
for y,z in zip(ys, M_CO_R):
    print('For', y, 'the recovery ratio was', round(np.nanmean(z),5))



#slope/aspect function 
Slope, Aspect = slopeAspect(Arry_DEM , 90)
#reclass Aspect function 
Aspect_R = reclassAspect( Aspect )
#reclass function: 10 classes
DEM_R = reclassByHisto(Slope, 10) 

#Call function to find zonal stats and export as a csv file
S_DEM= zonal_stats(DEM_R, mean_CO, 'Slope.csv')
A_DEM =zonal_stats(Aspect_R, mean_CO, 'Aspect.csv') 
#Export the coefficient of recovery as a GeoTiff
with rasterio.open(r'C:\\Users\\Aubre_M\\Documents\\Fall 2020\\GIS Programming_ Automation\\geog_4092\\Week 8\\data\\data\\fire_perimeter.tif') as dataset:
    with rasterio.open(f'C:\\Users\\Aubre_M\\Documents\\Fall 2020\\GIS Programming_ Automation\\geog_4092\\Week 8\\data\\data\\Recovery_Coefficents.tif' , 'w',
                       driver='GTiff',
                       height=mean_CO.shape[0],
                       width=mean_CO.shape[1],
                       count=1,
                       dtype=mean_CO.dtype,
                       crs=dataset.crs,
                       transform=dataset.transform, 
                       nodata=dataset.nodata
                      ) as out_dataset:
        out_dataset.write(mean_CO,1)
        
print('\n'+ 'The area showed that there is a greater relationship between the the recovery of vegation and the terrain'+
     'demonstrated that areas with a Northeast aspect and slope of less than 50 degrees has a greater recovery of vegeation')
print('\n', S_DEM)
print('\n',A_DEM)

The coeficient of recovery across all years was 0.02179563271729749
For 2002 the recovery ratio was 0.41127
For 2003 the recovery ratio was 0.54127
For 2004 the recovery ratio was 0.51346
For 2005 the recovery ratio was 0.61525
For 2006 the recovery ratio was 0.71617
For 2007 the recovery ratio was 0.70541
For 2008 the recovery ratio was 0.73951
For 2009 the recovery ratio was 0.71263
For 2010 the recovery ratio was 0.5851
For 2011 the recovery ratio was 0.62589

The area showed that there is a greater relationship between the the recovery of vegation and the terraindemonstrated that areas with a Northeast aspect and slope of less than 50 degrees has a greater recovery of vegeation

    Zone       Min       Max      Mean      Stdv  Count
0     1 -0.062426  0.103512  0.024422  0.025668  16811
1     2 -0.127041  0.233357  0.050322  0.055068  33138
2     3 -0.183811  0.318006  0.068162  0.075105  31116
3     4 -0.187782  0.369258  0.074374  0.088601  24044
4     5 -0.176336  0.463551  0.0